In [0]:
#я не торч, просто дунул
import torch
import sklearn.datasets

In [0]:
#воспроизводимость
import random
import numpy as np

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [0]:
wine_dataset = sklearn.datasets.load_wine()


In [0]:
from sklearn.model_selection import train_test_split
#используем для начала только два признака
X_train, X_test, y_train, y_test = train_test_split(wine_dataset.data[:, :2],
                                                    wine_dataset.target,
                                                    test_size=.3,
                                                    shuffle=True)
print(X_train.shape, y_train.shape, sep='\n')
print(np.unique(y_train))

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

(124, 2)
(124,)
[0 1 2]


In [0]:
class ClassificationNet(torch.nn.Module):
  def __init__(self, n_hidden_neurons, n_epochs=3000, lr=.001, batch_size=100):
    super(ClassificationNet, self).__init__()
    self.fc1 = torch.nn.Linear(2, n_hidden_neurons)
    self.act1 = torch.nn.Sigmoid()
    self.fc2 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
    self.act2 = torch.nn.Sigmoid()
    self.fc3 = torch.nn.Linear(n_hidden_neurons, 3)
    self.sm = torch.nn.Softmax(dim=1)
    
    self.n_epochs = n_epochs
    self.lr = lr
    self.batch_size = batch_size
 
  def forward(self, x):
    x = self.fc1(x)
    x = self.act1(x)
    x = self.fc2(x)
    x = self.act2(x)
    x = self.fc3(x)
    return x
  
  def train(self, x, y):
    loss = torch.nn.CrossEntropyLoss() #ждет НЕ Softmax выход (убран логарифм)
    optimizer = torch.optim.Adam(self.parameters(), self.lr=.001)
    
    #batch gd
    for _ in range(self.n_epochs):
      shuffled_indexes = np.random.permutation(x.shape[0])
      
      for start_index in range(0, len(shuffled_indexes), self.batch_size):
        x_batch = x[shuffled_indexes[start_index:start_index+self.batch_size]]
        y_batch = y[shuffled_indexes[start_index:start_index+self.batch_size]]
        
        optimizer.zero_grad()
        y_output = self.forward(x_batch)
        loss_val = loss(y_output, y_batch)
        loss_val.backward()
        optimizer.step()
        
    return self
  
  def predict(self, x):
    return self.forward(x).argmax(dim=1)
  
  def predict_proba(self, x):
    return self.sm(self.forward(x))

In [0]:
wine_net = ClassificationNet(30, batch_size=10)
wine_net.train(X_train, y_train)


ClassificationNet(
  (fc1): Linear(in_features=2, out_features=30, bias=True)
  (act1): Sigmoid()
  (fc2): Linear(in_features=30, out_features=30, bias=True)
  (act2): Sigmoid()
  (fc3): Linear(in_features=30, out_features=3, bias=True)
  (sm): Softmax()
)

In [0]:
from sklearn.metrics import accuracy_score

prediction = wine_net.predict(X_test)
accuracy_score(prediction, y_test)

0.8333333333333334